In [13]:
import mwclient
from transformers import pipeline
import time
from statistics import mean
import pandas as pd
from datetime import datetime

In [2]:
site = mwclient.Site("en.wikipedia.org")
page = site.pages["Bitcoin"]

In [3]:
revs = list(page.revisions())

In [4]:
revs[0]

OrderedDict([('revid', 1191519342),
             ('parentid', 1191352132),
             ('user', 'Jtbobwaysf'),
             ('timestamp',
              time.struct_time(tm_year=2023, tm_mon=12, tm_mday=24, tm_hour=1, tm_min=34, tm_sec=13, tm_wday=6, tm_yday=358, tm_isdst=-1)),
             ('comment', 'move down per talk')])

In [5]:
revs = sorted(revs, key=lambda rev: rev["timestamp"])

In [6]:
revs[0]

OrderedDict([('revid', 275832581),
             ('parentid', 0),
             ('user', 'Pratyeka'),
             ('timestamp',
              time.struct_time(tm_year=2009, tm_mon=3, tm_mday=8, tm_hour=16, tm_min=41, tm_sec=7, tm_wday=6, tm_yday=67, tm_isdst=-1)),
             ('comment', 'creation (stub)')])

In [7]:
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [8]:
def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

In [9]:
edits = {}

for rev in revs:
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)

    edits[date]["edit_count"] += 1

    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len(
            [s for s in edits[key]["sentiments"] if s < 0]
        ) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0

    del edits[key]["sentiments"]

In [10]:
edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [11]:
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,4,-0.550525,0.75
2009-08-05,1,0.748121,0.00
2009-08-06,2,0.995746,0.00
2009-08-14,1,0.930021,0.00
2009-10-13,2,-0.227501,0.50
...,...,...,...
2023-12-05,1,-0.995511,1.00
2023-12-06,2,-0.000489,0.50
2023-12-22,1,-0.990763,1.00
2023-12-23,1,0.973699,0.00


In [12]:
edits_df.index = pd.to_datetime(edits_df.index)

In [14]:
dates = pd.date_range(start="2009-03-08", end=datetime.today())

In [15]:
edits_df = edits_df.reindex(dates, fill_value=0)

In [16]:
rolling_edits = edits_df.rolling(30).mean()

In [17]:
rolling_edits

,edit_count,sentiment,neg_sentiment
2009-03-08,NaN,NaN,NaN
2009-03-09,NaN,NaN,NaN
2009-03-10,NaN,NaN,NaN
2009-03-11,NaN,NaN,NaN
2009-03-12,NaN,NaN,NaN
...,...,...,...
2023-12-26,1.100000,-0.161640,0.245139
2023-12-27,1.000000,-0.145745,0.222917
2023-12-28,0.466667,-0.147558,0.208333
2023-12-29,0.333333,-0.130530,0.183333


In [18]:
rolling_edits = rolling_edits.dropna()
rolling_edits

,edit_count,sentiment,neg_sentiment
2009-04-06,0.133333,-0.018351,0.025000
2009-04-07,0.000000,0.000000,0.000000
2009-04-08,0.000000,0.000000,0.000000
2009-04-09,0.000000,0.000000,0.000000
2009-04-10,0.000000,0.000000,0.000000
...,...,...,...
2023-12-26,1.100000,-0.161640,0.245139
2023-12-27,1.000000,-0.145745,0.222917
2023-12-28,0.466667,-0.147558,0.208333
2023-12-29,0.333333,-0.130530,0.183333


In [19]:
rolling_edits.to_csv("wikipedia_edits.csv")